In [ ]:
# downloading gcsfuse
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt -y -q update
!sudo apt -y -q install gcsfuse

deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  72485      0 --:--:-- --:--:-- --:--:-- 72485
OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5,385 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://packages.cloud.google.com/apt gcsfuse-bionic/main amd64 Packages [339 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!mkdir -p data
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 waymo_open_dataset_v_1_2_0_individual_files data

2021/06/16 02:22:26.843194 Using mount point: /content/data
2021/06/16 02:22:26.850414 Opening GCS connection...
2021/06/16 02:22:27.052098 Mounting file system "waymo_open_dataset_v_1_2_0_individual_files"...
2021/06/16 02:22:27.052621 File system has been successfully mounted.


In [ ]:
!pip install --upgrade pip
!pip install waymo-open-dataset-tf-2-4-0

     |████████████████████████████████| 1.6MB 5.1MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 3.1 MB 6.7 MB/s 
     |████████████████████████████████| 394.7 MB 16 kB/s 
     |████████████████████████████████| 462 kB 62.2 MB/s 
     |████████████████████████████████| 3.8 MB 44.6 MB/s 
     |████████████████████████████████| 2.9 MB 35.5 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Fou

In [ ]:
import tensorflow as tf
import os
from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils

In [ ]:
FILEPATH = {
    'train': './data/training',
    'test': './data/testing',
    'val': './data/validation',
}

In [ ]:
def getdataSet(filepath):
    dirs = os.listdir(filepath)
    dirs = [os.path.join(filepath,dir) for dir in dirs]

    raw_dataset = tf.data.TFRecordDataset(dirs)

    return raw_dataset

In [ ]:
dataSets = {}
for key in FILEPATH.keys():
    dataSets[key] = getdataSet(FILEPATH[key])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
HEIGHT, WIDTH = (640, 960)

In [ ]:
folder_path = {
    'train':'./train',
    'val':'./val'
}

for key in folder_path.keys():
    if not os.path.exists(folder_path[key]):
        os.mkdir(folder_path[key])

In [ ]:
def box2yoloFormat(center_x, center_y, length, width, ORIGIN_WIDTH, ORIGIN_HEIGHT):
    box_center_x = center_x / ORIGIN_WIDTH
    box_center_y = center_y / ORIGIN_HEIGHT
    box_width = length / ORIGIN_WIDTH
    box_height = width / ORIGIN_HEIGHT

    return (box_center_x, box_center_y, box_width, box_height)

In [ ]:
def imageDecode(image):
    
    image = tf.image.decode_jpeg(image)   

    h, w = image.shape[:2]

    image = tf.image.resize(image, (HEIGHT, WIDTH)) 
    images = tf.expand_dims(image, axis=0) / 255.0

    return images, h, w

In [ ]:
'''
enum Name {
    UNKNOWN = 0;
    FRONT = 1;
    FRONT_LEFT = 2;
    FRONT_RIGHT = 3;
    SIDE_LEFT = 4;
    SIDE_RIGHT = 5;
  }
'''
def createImgLabel(dataSetType):
    # SAVE FILE
    n = 0
    for record in dataSets[dataSetType]:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(record.numpy()))
        
        
        
        for i in range(5):
            camera_image = frame.images[i]
            if camera_image.name in [1,2,3]:
                if n%10 == 0:
                    print(n)
                if n <= 88378:
                    n = n + 1
                    continue

                # IMAGE
                images, ORIGIN_HEIGHT, ORIGIN_WIDTH = imageDecode(frame.images[i].image)
                plt.imsave( f'{folder_path[dataSetType]}/{n}.jpg', images[0].numpy())    
               
                # BOUNDING BOX
                for camera_labels in frame.camera_labels:
                    # Ignore camera labels that do not correspond to this camera.
                    if camera_labels.name != camera_image.name:
                        continue
                      
                    boxs = []
                    for label in camera_labels.labels:        
                        # BOX
                        box = label.box        
                        box_bound = box2yoloFormat(box.center_x, box.center_y, box.length, box.width, ORIGIN_WIDTH, ORIGIN_HEIGHT)
                        
                        # LABEL
                        box_label = label.type

                        boxs.append((box_label, *box_bound))
                    
                    with open(f'{folder_path[dataSetType]}/{n}.txt','w') as f:
                        box_str = []
                        for box in boxs:         
                            box_str.append(' '.join(str(x) for x in box))
                        f.write('\n'.join(box_str))
                
                n = n + 1
           
    

    

In [ ]:
def writeImgFileList(dataSetType):
    # WRITE train.txt
    with open(f'/content/waymo_object_detection/cfg/{dataSetType}.txt', 'w') as out:
        for img in [f for f in os.listdir(f'/content/waymo_object_detection/{dataSetType}') if f.endswith('jpg')]:
            out.write(f'/content/waymo_object_detection/{dataSetType}/' + img + '\n')

In [ ]:
createImgLabel('train')

串流輸出內容已截斷至最後 5000 行。
43310
43320
43330
43340
43350
43360
43370
43380
43390
43400
43410
43420
43430
43440
43450
43460
43470
43480
43490
43500
43510
43520
43530
43540
43550
43560
43570
43580
43590
43600
43610
43620
43630
43640
43650
43660
43670
43680
43690
43700
43710
43720
43730
43740
43750
43760
43770
43780
43790
43800
43810
43820
43830
43840
43850
43860
43870
43880
43890
43900
43910
43920
43930
43940
43950
43960
43970
43980
43990
44000
44010
44020
44030
44040
44050
44060
44070
44080
44090
44100
44110
44120
44130
44140
44150
44160
44170
44180
44190
44200
44210
44220
44230
44240
44250
44260
44270
44280
44290
44300
44310
44320
44330
44340
44350
44360
44370
44380
44390
44400
44410
44420
44430
44440
44450
44460
44470
44480
44490
44500
44510
44520
44530
44540
44550
44560
44570
44580
44590
44600
44610
44620
44630
44640
44650
44660
44670
44680
44690
44700
44710
44720
44730
44740
44750
44760
44770
44780
44790
44800
44810
44820
44830
44840
44850
44860
44870
44880
44890
44900
44910
44920
44930
4

KeyboardInterrupt: ignored

In [ ]:
writeImgFileList('train')

In [ ]:
createImgLabel('val')

In [ ]:
!mv /content/drive/MyDrive/課程相關/109-2/自動駕駛實務/專案/Final/train/train/* /content/drive/MyDrive/課程相關/109-2/自動駕駛實務/專案/Final/train